### Extract features from the processed and augmented audio files.

In [1]:
# Imports
import os
import random
import logging
import warnings
import gc

import numpy as np
import pandas as pd
from tqdm import tqdm


# Data processing and scientific computing
from scipy.io import wavfile
from scipy.signal import butter, lfilter
from scipy.spatial.distance import cosine

# Audio processing
import librosa
import soundfile as sf

# Visualization
import matplotlib.pyplot as plt
import plotly.express as px

# Set up logging
logging.basicConfig(level=logging.INFO)

In [2]:
# Load the prepared csv
augmented_data = pd.read_csv("augmented_data.csv")

In [3]:
augmented_data.head()

,genus,species,latitude,longitude,quality,file_name,simplified_type,season,time_of_day,length_seconds,processed_file,augmentations
0,Branta,canadensis,39.2095,-84.7821,A,Branta_canadensis_Whitewater_Township_near__Ha...,Call,Spring,Morning,14,Branta_canadensis_Whitewater_Township_near__Ha...,NaN
1,Branta,canadensis,39.2095,-84.7821,A,Branta_canadensis_Whitewater_Township_near__Ha...,Call,Spring,Morning,14,Branta_canadensis_Whitewater_Township_near__Ha...,NaN
2,Branta,canadensis,39.2095,-84.7821,A,Branta_canadensis_Whitewater_Township_near__Ha...,Call,Spring,Morning,14,Branta_canadensis_Whitewater_Township_near__Ha...,NaN
3,Branta,canadensis,39.2095,-84.7821,A,Branta_canadensis_Whitewater_Township_near__Ha...,Call,Spring,Morning,14,Branta_canadensis_Whitewater_Township_near__Ha...,NaN
4,Branta,canadensis,39.2095,-84.7821,A,Branta_canadensis_Whitewater_Township_near__Ha...,Call,Spring,Morning,14,Branta_canadensis_Whitewater_Township_near__Ha...,NaN


In [4]:
def is_valid_audio(audio, sr, min_duration=0.1, silence_threshold=-60):
    """Check if the audio segment is valid (not too short and not silent)."""
    duration = librosa.get_duration(y=audio, sr=sr)
    if duration < min_duration:
        return False
    # Check if the audio is mostly silent
    db = librosa.amplitude_to_db(np.abs(audio), ref=np.max)
    if np.mean(db) < silence_threshold:
        return False
    return True

def extract_features(audio, sr):
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(audio)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)[0]
    return {
        'mfccs': mfccs,
        'spectral_centroids': spectral_centroids,
        'chroma': chroma,
        'zero_crossing_rate': zero_crossing_rate,
        'spectral_rolloff': spectral_rolloff
    }

def summarize_feature(feature):
    if feature.ndim == 1:
        return np.array([np.mean(feature), np.std(feature), np.max(feature)])
    elif feature.ndim == 2:
        return np.hstack([
            np.mean(feature, axis=1),
            np.std(feature, axis=1),
            np.max(feature, axis=1)
        ])
    else:
        raise ValueError(f"Unsupported feature dimension: {feature.ndim}")


In [5]:
def process_audio_file(file_path):
    try:
        # Load audio file
        audio, sr = librosa.load(file_path, sr=None)
        
        # Check if audio segment is valid
        if not is_valid_audio(audio, sr):
            print(f"Warning: Audio file {file_path} is too short or silent. Skipping.")
            return None

        # Extract features
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            features = extract_features(audio, sr)
        
        # Summarize features
        feature_summary = {}
        for key, value in features.items():
            feature_summary[f"{key}_summary"] = summarize_feature(value)
        
        # Create feature vector
        feature_vector = np.hstack([
            feature_summary['mfccs_summary'],
            feature_summary['spectral_centroids_summary'],
            feature_summary['chroma_summary'],
            feature_summary['zero_crossing_rate_summary'],
            feature_summary['spectral_rolloff_summary']
        ])
        
        return feature_vector
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return None

def process_audio_files_batch(df, base_dir, batch_size=500):
    feature_data = []
    skipped_files = []
    
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        batch_feature_data = []  # Temporary storage for batch features
        for _, row in tqdm(batch.iterrows(), total=len(batch)):
            possible_paths = [
                os.path.join(base_dir, 'Augmented Recordings', row['processed_file']),
                os.path.join(base_dir, 'Processed Recordings', row['processed_file'])
            ]
            
            file_path = next((path for path in possible_paths if os.path.exists(path)), None)
            
            if file_path is None:
                print(f"File not found: {row['processed_file']}")
                skipped_files.append(row['processed_file'])
                continue
            
            feature_vector = process_audio_file(file_path)
            
            if feature_vector is not None:
                batch_feature_data.append({
                    'processed_file': row['processed_file'],
                    'feature_vector': feature_vector,
                })
            else:
                skipped_files.append(row['processed_file'])
            
            # More frequent garbage collection to avoid memory overflow
            gc.collect()
        
        # Append batch data to main feature data and save to disk
        feature_data.extend(batch_feature_data)
        batch_df = pd.DataFrame(batch_feature_data)
        batch_df.to_csv(f"feature_data_batch_{i//batch_size}.csv", index=False)
        
        # Clear batch data from memory
        batch_feature_data.clear()
        gc.collect()

    print(f"Total files skipped: {len(skipped_files)}")
    return feature_data, skipped_files

In [6]:
# Usage
base_dir = os.getcwd()

feature_data, skipped_files = process_audio_files_batch(augmented_data, base_dir)

# Convert feature_data to DataFrame
feature_df = pd.DataFrame(feature_data)

# Merge the new feature DataFrame with the existing final_data DataFrame
final_data = pd.merge(augmented_data, feature_df, on='processed_file', how='left')

# Drop rows corresponding to skipped files
final_data = final_data[~final_data['processed_file'].isin(skipped_files)]

print(final_data.info())

print(f"\nTotal files in augmented_data: {len(augmented_data)}")
print(f"Files successfully processed: {len(feature_df)}")
print(f"Files skipped (too short or silent): {len(skipped_files)}")
print(f"Files in final_data after dropping skipped files: {len(final_data)}")

  6%|▌         | 28/500 [00:05<00:57,  8.24it/s]

 23%|██▎       | 113/500 [00:17<00:49,  7.80it/s]

 24%|██▍       | 122/500 [00:19<00:49,  7.56it/s]

 30%|██▉       | 148/500 [00:26<01:37,  3.60it/s]

 34%|███▎      | 168/500 [00:32<01:24,  3.93it/s]

...

 52%|█████▏    | 91/174 [00:13<00:12,  6.79it/s]

 76%|███████▌  | 132/174 [00:18<00:04,  8.61it/s]

 82%|████████▏ | 143/174 [00:20<00:03,  8.36it/s]

100%|██████████| 174/174 [00:24<00:00,  7.16it/s]


Total files skipped: 1437
<class 'pandas.core.frame.DataFrame'>
Index: 46737 entries, 0 to 48173
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   genus            46737 non-null  object 
 1   species          46737 non-null  object 
 2   latitude         46737 non-null  float64
 3   longitude        46737 non-null  float64
 4   quality          46737 non-null  object 
 5   file_name        46737 non-null  object 
 6   simplified_type  46737 non-null  object 
 7   season           46737 non-null  object 
 8   time_of_day      46737 non-null  object 
 9   length_seconds   46737 non-null  int64  
 10  processed_file   46737 non-null  object 
 11  augmentations    35019 non-null  object 
 12  feature_vector   46737 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 5.0+ MB
None

Total files in augmented_data: 48174
Files successfully processed: 46737
Files skipped (too short or silent): 1

In [7]:
# Review the dataframe
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46737 entries, 0 to 48173
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   genus            46737 non-null  object 
 1   species          46737 non-null  object 
 2   latitude         46737 non-null  float64
 3   longitude        46737 non-null  float64
 4   quality          46737 non-null  object 
 5   file_name        46737 non-null  object 
 6   simplified_type  46737 non-null  object 
 7   season           46737 non-null  object 
 8   time_of_day      46737 non-null  object 
 9   length_seconds   46737 non-null  int64  
 10  processed_file   46737 non-null  object 
 11  augmentations    35019 non-null  object 
 12  feature_vector   46737 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 5.0+ MB


In [10]:
final_data = final_data.fillna("None")
final_data.head()

,genus,species,latitude,longitude,quality,file_name,simplified_type,season,time_of_day,length_seconds,processed_file,augmentations,feature_vector
0,Branta,canadensis,39.2095,-84.7821,A,Branta_canadensis_Whitewater_Township_near__Ha...,Call,Spring,Morning,14,Branta_canadensis_Whitewater_Township_near__Ha...,None,"[-255.54920959472656, 142.5915069580078, -49.8..."
1,Branta,canadensis,39.2095,-84.7821,A,Branta_canadensis_Whitewater_Township_near__Ha...,Call,Spring,Morning,14,Branta_canadensis_Whitewater_Township_near__Ha...,None,"[-267.11663818359375, 140.6734161376953, -51.6..."
2,Branta,canadensis,39.2095,-84.7821,A,Branta_canadensis_Whitewater_Township_near__Ha...,Call,Spring,Morning,14,Branta_canadensis_Whitewater_Township_near__Ha...,None,"[-274.216552734375, 131.9573974609375, -51.501..."
3,Branta,canadensis,39.2095,-84.7821,A,Branta_canadensis_Whitewater_Township_near__Ha...,Call,Spring,Morning,14,Branta_canadensis_Whitewater_Township_near__Ha...,None,"[-270.9906921386719, 134.13259887695312, -53.0..."
4,Branta,canadensis,39.2095,-84.7821,A,Branta_canadensis_Whitewater_Township_near__Ha...,Call,Spring,Morning,14,Branta_canadensis_Whitewater_Township_near__Ha...,None,"[-332.7778015136719, 114.2027816772461, -45.52..."


In [11]:
# Save DataFrame to CSV
final_data.to_csv('tabular_data.csv', index=False)

print("Processing complete. Summary data saved to 'tabular_data.csv'.")

Processing complete. Summary data saved to 'tabular_data.csv'.
